In [114]:
import pandas as pd
# produce "data/all_data.csv" for later usage

df_gdp = pd.read_csv("data_gdp\gdp_data.csv")
df_wdi = pd.read_csv("data\cleanWDI.csv")
df_wdi.loc[df_wdi["Country Name"] == "Namibia", "Country Code"] = "NB"
# df_wdi[df_wdi["Country Name"] == "Namibia"]

<>:4: SyntaxWarning: invalid escape sequence '\g'
<>:5: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\g'
<>:5: SyntaxWarning: invalid escape sequence '\c'
C:\Users\tvuontis\AppData\Local\Temp\ipykernel_20080\2092190555.py:4: SyntaxWarning: invalid escape sequence '\g'
  df_gdp = pd.read_csv("data_gdp\gdp_data.csv")
C:\Users\tvuontis\AppData\Local\Temp\ipykernel_20080\2092190555.py:5: SyntaxWarning: invalid escape sequence '\c'
  df_wdi = pd.read_csv("data\cleanWDI.csv")


In [115]:
from database.utils import code_based_get, get_country
from tqdm import tqdm

def get_a2_map(df):
    mapping = {}
    failed = []
    for c in tqdm(df["name"].unique(), "Solving alpha2 values"):
        try:
            if c == "Namibia":
                mapping[c] == "NB"
            else:
                mapping[c] = get_country(c).alpha_2
        except:
            try:
                mapping[c] = code_based_get(c).alpha_2
            except:
                failed.append(c)
    if failed:
        print(failed)
        raise KeyError(f"{', '.join(failed)} not found by get_country")
    return mapping

In [116]:
df_gdp.rename(columns={"Country Name": "name", "Time": "year"}, inplace=True)
df_wdi.rename(columns={"Country Code": "alpha_2", "Time": "year", "Country Name": "name"}, inplace=True)

In [117]:
mapping = get_a2_map(df_gdp)
print(mapping)

Solving alpha2 values: 100%|██████████| 266/266 [00:03<00:00, 82.45it/s]

{'Aruba': 'AW', 'Africa Eastern and Southern': 'XX', 'Afghanistan': 'AF', 'Africa Western and Central': 'XX', 'Angola': 'AO', 'Albania': 'AL', 'Andorra': 'AD', 'Arab World': 'XX', 'United Arab Emirates': 'AE', 'Argentina': 'AR', 'Armenia': 'AM', 'American Samoa': 'AS', 'Antigua and Barbuda': 'AG', 'Australia': 'AU', 'Austria': 'AT', 'Azerbaijan': 'AZ', 'Burundi': 'BI', 'Belgium': 'BE', 'Benin': 'BJ', 'Burkina Faso': 'BF', 'Bangladesh': 'BD', 'Bulgaria': 'BG', 'Bahrain': 'BH', 'Bahamas, The': 'XX', 'Bosnia and Herzegovina': 'BA', 'Belarus': 'BY', 'Belize': 'BZ', 'Bermuda': 'BM', 'Bolivia': 'BO', 'Brazil': 'BR', 'Barbados': 'BB', 'Brunei Darussalam': 'BN', 'Bhutan': 'BT', 'Botswana': 'BW', 'Central African Republic': 'CF', 'Canada': 'CA', 'Central Europe and the Baltics': 'XX', 'Switzerland': 'CH', 'Channel Islands': 'XX', 'Chile': 'CL', 'China': 'CN', "Cote d'Ivoire": 'CI', 'Cameroon': 'CM', 'Congo, Dem. Rep.': 'XX', 'Congo, Rep.': 'XX', 'Colombia': 'CO', 'Comoros': 'KM', 'Cabo Verde': 

In [118]:
# Convert ISO2_Code to alpha_2
df_gdp.insert(loc = 2, column = "alpha_2", value = df_gdp["name"].map(mapping))
df_gdp.drop(["ISO2_code"], axis=1, inplace=True)
df_gdp.head(5)

,name,alpha_2,year,GDP
0,Aruba,AW,1960,NaN
1,Aruba,AW,1961,NaN
2,Aruba,AW,1962,NaN
3,Aruba,AW,1963,NaN
4,Aruba,AW,1964,NaN


In [119]:
df = pd.merge(df_gdp, df_wdi, on=["alpha_2", "year"], how="outer")
df.dropna(subset=['GDP'],inplace=True)
df = df[df.alpha_2 != "XX"]
df["alpha_2"] = df["alpha_2"].fillna("NA")
df.shape

(10447, 21)

In [120]:
rename_map = {
    'Fertility rate, total (births per woman) [SP.DYN.TFRT.IN]': 'fertility_rate',
    'Birth rate, crude (per 1,000 people) [SP.DYN.CBRT.IN]': 'birth_rate',
    'Death rate, crude (per 1,000 people) [SP.DYN.CDRT.IN]': 'death_rate',
    'Life expectancy at birth, total (years) [SP.DYN.LE00.IN]': 'life_expectancy',
    'Net migration [SM.POP.NETM]': 'net_migration',
    'Individuals using the Internet (% of population) [IT.NET.USER.ZS]': 'internet_users_pct',
    'Gini index [SI.POV.GINI]': 'gini_index',
    'Human capital index (HCI) (scale 0-1) [HD.HCI.OVRL]': 'human_capital_index',
    'School enrollment, secondary (gross), gender parity index (GPI) [SE.ENR.SECO.FM.ZS]': 'school_enroll_secondary_gpi',
    'Literacy rate, youth (ages 15-24), gender parity index (GPI) [SE.ADT.1524.LT.FM.ZS]': 'youth_literacy_gpi',
    'Urban population (% of total population) [SP.URB.TOTL.IN.ZS]': 'urban_population_pct',
    'Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]': 'infant_mortality_rate',
    'Current health expenditure per capita, PPP (current international $) [SH.XPD.CHEX.PP.CD]': 'health_expenditure_per_capita_ppp',
    'Population, female (% of total population) [SP.POP.TOTL.FE.ZS]': 'female_population_pct',
    'Population, male (% of total population) [SP.POP.TOTL.MA.ZS]': 'male_population_pct',
    'Population, total [SP.POP.TOTL]': 'population_WDI',
    'pop': 'population_WRP',
    'population': 'population_WPP',
}
df.rename(columns=rename_map, inplace=True)

In [121]:
# Remove countries where names differ
mask = df["name_x"] != df["name_y"]
df = df[~mask]
df[df["name_x"] != df["name_y"]]


,name_x,alpha_2,year,GDP,name_y,fertility_rate,birth_rate,death_rate,life_expectancy,net_migration,...,gini_index,human_capital_index,school_enroll_secondary_gpi,youth_literacy_gpi,urban_population_pct,infant_mortality_rate,health_expenditure_per_capita_ppp,female_population_pct,male_population_pct,population_WDI


In [122]:
df.rename(columns={"name_x": "name",}, inplace=True)

columns_of_interest = [
    'name',
    'alpha_2',
    'year',
    'population_WDI',
    'GDP',
    'life_expectancy',
    'net_migration',
    'internet_users_pct',
    'human_capital_index',
    'school_enroll_secondary_gpi',
    'youth_literacy_gpi',
    'urban_population_pct',
    'infant_mortality_rate',
    'female_population_pct',
    'male_population_pct',
    ]
df = df[[col for col in columns_of_interest if col in df.columns]]
df.to_csv("data/wdi_gdp_data.csv")